In [1]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor 
import matplotlib.pyplot as plt
import os.path
from os import path

In [2]:
def LoadRawVariables():
    c2pt = []
    ts   = []
    taus = []
    xs   = []
    ys   = []
    zs   = []
    c3pt = []
    
    c2pt_start = 9
    c3pt_start = 10155
    
    for tau in range(0, 49, 8):
        for x in range(0, 25, 8):
            for y in range(0, 25, 8):
                for z in range(0, 25, 8):
                    for sample in range(748, 1421, 16):
                        fname = "../Data/T" + str(tau) + "/x" + str(x) + "y" + str(y) + "z" + str(z) + "/nuc3pt.dat." + str(sample)
                        if path.exists(fname):
                            with open(fname) as fp:
                                for i, line in enumerate(fp):
                                    if i >= 7 and i <= 70:
                                        c2pt.append([float(x) for x in line.rstrip().split()[1:3]])
                                        ts.append(i - 7)
                                        taus.append(tau)
                                        xs.append(x)
                                        ys.append(y)
                                        zs.append(z)
                                    elif i >= 10154 and i <= 10217:
                                        c3pt.append([float(x) for x in line.rstrip().split()[1:3]])
                                    elif i > 10217:
                                        break
    
    return ts, taus, xs, ys, zs, c2pt, c3pt

ts, taus, xs, ys, zs, c2pt, c3pt = LoadRawVariables()

In [38]:
features = np.array([np.array([ts[i], taus[i], xs[i], ys[i], zs[i], c2pt[i][0], c2pt[i][1]]) for i in range(len(ts))])
labels = np.array([np.array([c3pt[i][0], c3pt[i][1]]) for i in range(len(c3pt))])

In [39]:
labelFrac = 0.9
BCFrac = 0.05

labelEnd = int(len(labels) * labelFrac)
BCEnd    = int(len(labels) * (BCFrac + labelFrac))

X_train, Y_train, X_bc, Y_bc, X_test, Y_test = features[:labelEnd], labels[:labelEnd], features[labelEnd:BCEnd], labels[labelEnd:BCEnd], features[BCEnd:], labels[BCEnd:]

gbr_real = GradientBoostingRegressor(learning_rate=0.1, n_estimators=100, max_depth=3)
gbr_real.fit(X_train, Y_train[:, 0])

y_bc_pred_real = gbr_real.predict(X_bc)

biasCrxn_real = np.average(Y_bc[:, 0] - y_bc_pred_real)

gbr_imaginary = GradientBoostingRegressor(learning_rate=0.1, n_estimators=100, max_depth=3)
gbr_imaginary.fit(X_train, Y_train[:, 1])

y_bc_pred_imaginary = gbr_imaginary.predict(X_bc)

biasCrxn_imaginary = np.average(Y_bc[:, 1] - y_bc_pred_imaginary)

In [41]:
print(biasCrxn_real)
print(biasCrxn_imaginary)

-49647498599.08556
-69320103552.21843


In [44]:
trials = 0
error = 0
for i in range(len(X_test)):
    testImg = X_test[i]
    testLabel = Y_test[i]
    pred_real = gbr_real.predict([testImg]) + biasCrxn_real
    pred_imaginary = gbr_imaginary.predict([testImg]) + biasCrxn_imaginary
    error += np.sqrt((pred_real - testLabel[0]) ** 2 + (pred_imaginary - testLabel[1]) ** 2)
    trials += 1
error /= trials
print(error)

[9.38057294e+11]
